In [ ]:
import dbnl
import pandas as pd
import json
from datetime import UTC, datetime, timedelta

# Make sure your version matches the docs at https://docs.dbnl.com/
print("dbnl version:", dbnl.__version__)

In [ ]:
# Login to DBNL (using default Sandbox url)
dbnl.login(
    api_url="http://localhost:8080/api",
    api_token="<DBNL_API_KEY>", # found at http://localhost:8080/tokens
)

In [ ]:
# Create a new project
project = dbnl.get_or_create_project(
    name="ADK Calc Traces SDK via OTEL file and augment",
    schedule="daily",  # How often DBNL analyzes new data
    default_llm_model_name="quickstart_model" # From step (2) in quickstart
)

In [ ]:
from dbnl_otel_converter import dbnl_df_from_otel_file

df = dbnl_df_from_otel_file("traces.jsonl")

print(f"Loaded {len(df)} traces.")

In [ ]:
# Add a total_cost column to our dataframe approximating the cost from input and output tokens

COST = { # per token
    "gen_ai.usage.input_tokens": 0.000000075,
    "gen_ai.usage.output_tokens": 0.00000030,
}

raw_spans = df["traces_data"]
dbnl_spans = dbnl.convert_otlp_traces_data(data=raw_spans)

def est_cost_from_gen_ai_tokens(spans):
    """Sum gen_ai.usage.input_tokens + gen_ai.usage.output_tokens across all spans."""
    if spans is None:
        return 0

    total = 0
    for span in spans:
        attrs = span.get("attributes", [])
        # attrs is a list of (key, value) tuples
        for key, val in attrs:
            if key in ("gen_ai.usage.input_tokens", "gen_ai.usage.output_tokens"):
                if isinstance(val, str):
                    # strip wrapping quotes if present, then try to parse int
                    v = val.strip('"')
                    try:
                        total += int(v)*COST[key]
                    except ValueError:
                        pass  # ignore non-numeric weirdness
    return total

df["total_cost"] = dbnl_spans.apply(est_cost_from_gen_ai_tokens)

In [ ]:
# if the data is spread across multiple days we need to break it into days for upload
df["date"] = df["timestamp"].dt.tz_convert("UTC").dt.date
day_dfs = [g.drop(columns="date").reset_index(drop=True)
           for _, g in df.groupby("date")]

print("Uploading data...")
print(f"See status at: http://localhost:8080/ns/{project.namespace_id}/projects/{project.id}/status")

for idx, day_df in enumerate(day_dfs):
    print(f"{idx + 1} / {len(day_dfs)} publishing log data for {min(day_df['timestamp']).date()}")
    data_start_t = min(day_df['timestamp']).replace(hour=0, minute=0, second=0, microsecond=0)
    data_end_t = data_start_t + timedelta(days=1)
    try:
        dbnl.log(
            project_id=project.id,
            data_start_time=data_start_t,
            data_end_time=data_end_t,
            data=day_df,
        )
    except Exception as e:
        if "Data already exists" in str(e):
            continue
        raise

print("You can now explore your data in DBNL!")
print(f"http://localhost:8080/ns/{project.namespace_id}/projects/{project.id}")